In [1]:
# syft absolute
import syft as sy
from syft.types.syft_object import SyftObject
from syft import ActionObject

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///tmp/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=6837160a42cf433595cc4b3df70c5d1a in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///tmp/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=5087b7cc70604f2c9f4d0299af221fa5 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
import inspect
from syft.service.code.user_code_service import UserCodeService
from syft.service.policy.policy import InputPolicy, OutputPolicy
from syft.service.request.request import Request, UserCodeStatusChange
from syft.service.code.user_code import UserCodeStatus, UserCode, UserCodeStatusCollection
from syft.store.linked_obj import LinkedObject
from syft.types.datetime import DateTime
from syft.types.uid import UID

def test():
    pass

request_id = UID()
user_code = UserCode(
    id = UID(),
    node_uid = node_low.python_node.id,
    user_verify_key = client_low.credentials.verify_key,
    raw_code = "",
    input_policy_type = InputPolicy,
    input_policy_init_kwargs = None,
    input_policy_state = b"",
    output_policy_type = OutputPolicy,
    output_policy_init_kwargs = None,
    output_policy_state = b"",
    parsed_code = '',
    service_func_name = '',
    unique_func_name = '',
    user_unique_func_name = '', 
    code_hash = '',
    signature = inspect.signature(test),
    status = UserCodeStatusCollection(status_dict={}),
    input_kwargs = []
)

linked_obj = LinkedObject(
    node_uid = node_low.python_node.id,
    service_type = UserCodeService,
    object_type = UserCode,
    object_uid = user_code.id,
)


low_request = Request(
    id=request_id,
    requesting_user_verify_key=client_low.credentials.verify_key,
    requesting_user_name="Teo",
    requesting_user_email= "teo@om.org",
    requesting_user_institution= "OM",
    approving_user_verify_key= client_low.credentials.verify_key,
    request_time=DateTime.now(),
    updated_at=DateTime.now(),
    node_uid=node_low.python_node.id,
    request_hash='a',
    changes=[UserCodeStatusChange(
            value=UserCodeStatus.PENDING,
            linked_obj=linked_obj,
        )],
    history=[],
)


In [5]:
# import json
# def request_to_json(request):
#     dict = {
#         'id': request.id,
#         "requesting_user_verify_key": ,
#         "requesting_user_name": request.requesting_user_name,
#         "requesting_user_email": request.requesting_user_email,
#         "requesting_user_institution": request.requesting_user_institution,
#         "approving_user_verify_key": ,
#         "request_time": ,
#         "updated_at": ,
#         "node_uid": ,
#         "request_hash": ,
#         "changes": ,
#         "history": ,
#     }
    
# def json_to_request(json):


In [6]:
high_request = Request(
    id=request_id,
    requesting_user_verify_key=client_low.credentials.verify_key,
    requesting_user_name="Teodor",
    requesting_user_email= "teodor@om.org",
    requesting_user_institution= "OpenMined",
    approving_user_verify_key= client_low.credentials.verify_key,
    request_time=DateTime.now(),
    updated_at=DateTime.now(),
    node_uid=node_low.python_node.id,
    request_hash='b',
    changes=[UserCodeStatusChange(
            value=UserCodeStatus.APPROVED,
            linked_obj=linked_obj,
        )],
    history=[],
)


In [7]:
from syft.client.syncing import get_diff_request, Diff
request_diff = Diff(
    low_obj=low_request,
    high_obj=high_request,
    obj_type=Request,
    merge_state="DIFF",
    diff_list=get_diff_request(low_request, high_request)
)


In [8]:
request_diff

syft.client.syncing.Diff

In [9]:
[request_diff]

[syft.client.syncing.Diff]

In [10]:
# assert False

# create datasets

In [11]:
# third party
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=private_low.shape,
            mock_is_real=True,
        )
    ],
)

client_low.upload_dataset(dataset_low)

  0%|          | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|██████████| 1/1 [00:00<00:00,  6.21it/s]


SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [12]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    id=dataset_low.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|██████████| 1/1 [00:00<00:00,  9.50it/s]


Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [13]:
data_low = client_low.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///tmp/320ee800d86c416fa83fc3b6dfe756a1.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=27f0c5c882e744a38d5b0a4ca85fd1b9 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_2182: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_2182
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [14]:
from time import sleep
sleep(1)

In [15]:
client_low.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [16]:
request = client_low.requests[0]

In [17]:
code = client_low.code[0]

TEO HERE

In [18]:
request.changes

[syft.service.request.request.UserCodeStatusChange]

In [19]:
from syft.client.syncing import get_sync_state
state = get_sync_state(client_low, client_high) # between two nodes

SyftInfo: Node Landed!


In [20]:
state.diff_to_sync

[syft.client.syncing.Diff, syft.client.syncing.Diff]

In [21]:
state.objs_to_sync()

error representing <class 'list'> of objects. All arrays must be of the same length, Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/syft_object.py", line 773, in list_dict_repr_html
    vals = get_repr_values_table(self, is_homogenous, extra_fields=extra_fields)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/syft_object.py", line 719, in get_repr_values_table
    df = pd.DataFrame(cols)
         ^^^^^^^^^^^^^^^^^^
  File "/home/teo/anaconda3/envs/syft_3.11/lib/python3.11/site-packages/pandas/core/frame.py", line 664, in __init__
    mgr = dict_to_mgr(data, index, columns, dtype=dtype, copy=copy, typ=manager)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/anaconda3/envs/syft_3.11/lib/python3.11/site-packages/pandas/core/internals/construction.py", line 493, in dict_to_mgr
    return array

[syft.service.request.request.Request, syft.service.code.user_code.UserCode]

In [22]:
# items_to_sync = [request, code]
items_to_sync = state.objs_to_sync()

- What do we do with properties here?
  - just set them to None

In [23]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [24]:
# job.node_uid

In [25]:
node_high.python_node.id

<UID: 8212e6797fde4c3fba4fc53ab555a886>

In [26]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        x.syft_node_location=node_uid
        x.syft_client_verify_key=verify_key
        if hasattr(x, "node_uid"):
            x.node_uid=node_uid
                    
                    
    # TODO: hadnle nested collections
            

In [27]:
def create_actionobject(action_object, client):
    action_object = action_object.refresh_object()
    res = client.api.services.action.set(action_object)
    print("Created", res)

In [28]:
def sync_items(items, node, client):
    for item in items_to_sync:
        
        if isinstance(item, ActionObject):
            create_actionobject(item, client)

        else:
            stash = get_stash_for_item(item, node)

            set_obj_ids(item, client.id, client.verify_key)

            res = stash.delete_by_uid(node.python_node.verify_key, item.id)

            res = stash.set(node.python_node.verify_key, item)
            if not res.is_ok():
                raise ValueError("")
            else:
                print("created", item)

In [29]:
sync_items(items_to_sync, node_high, client_high)

created syft.service.request.request.Request
created syft.service.code.user_code.UserCode


# Run code high and sync back result

In [30]:
client_high._fetch_api(client_high.credentials)

In [31]:
data_high = client_high.datasets[0].assets[0]

In [32]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = dd579fa9bcfe4025b2c983d02f2c6190
    status: JobStatus.CREATED
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [33]:
# wait for the result
job_high.wait()

```python
Pointer
```
17.0

In [34]:
job_high.result.get()

17.0

In [35]:
request_high = client_high.requests[0]
code = request_high.code
log = job_high._get_log_objs()

In [36]:
items_to_sync = [code, job_high.result, job_high, log]

In [37]:
state_2 = get_sync_state(client_low, client_high)

In [53]:
state_2.diff_to_sync

[syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff]

In [39]:
items_to_sync = state_2.objs_to_sync()
items_to_sync.append(job_high.result)

In [40]:
x = client_high.requests[0]

In [41]:
request_high.approve()

Approving request for domain test_h


"Cannot Modify Status as the Domain's data is not included in the request"

In [42]:
sync_items(items_to_sync, node_low, client_low)

Detached action object, object exists but is not linked to data in the blob storage SyftError: No blob storage entry exists for uid: e8c3070f1235477ba359090d663f1940
Detached action object, object exists but is not linked to data in the blob storage SyftError: No blob storage entry exists for uid: e8c3070f1235477ba359090d663f1940
created syft.service.job.job_stash.Job
Created SyftError: Arg: Could not find item with uid 857941e17ba54bc4bd27c522b8d365e0, "857941e17ba54bc4bd27c522b8d365e0 not in <class 'syft.store.sqlite_document_store.SQLiteBackingStore'>" must be Union not str
created syft.service.log.log.SyftLog
Created 17.0


In [ ]:
# low_node_state = client_low.get_sync_state() -> just a table
# high_node_state = client_high.get_sync_state()
# (low_new_state, high_new_state) = resolve_state(low_node_state, high_node_state)

In [43]:
state_3 = get_sync_state(client_low, client_high)

Detached action object, object exists but is not linked to data in the blob storage SyftError: No blob storage entry exists for uid: e8c3070f1235477ba359090d663f1940
Detached action object, object exists but is not linked to data in the blob storage SyftError: No blob storage entry exists for uid: e8c3070f1235477ba359090d663f1940


In [44]:
state_3.diff_to_sync

Detached action object, object exists but is not linked to data in the blob storage SyftError: No blob storage entry exists for uid: e8c3070f1235477ba359090d663f1940
Detached action object, object exists but is not linked to data in the blob storage SyftError: No blob storage entry exists for uid: e8c3070f1235477ba359090d663f1940


[syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff]

In [45]:
state_3.diff_to_sync[1]

syft.client.syncing.Diff

# Run code low

In [46]:
res_low = client_low.code.compute_mean(data=data_low)

In [47]:
# we need to sync the usercode itself


In [48]:
res_low.get()

7.0

In [49]:
private_high.mean()

17.0

In [50]:
job_low = client_low.code.compute_mean(data=data_low, blocking=False)

SyftWarning: There are existing jobs for this user code, returning the latest one

In [51]:
job_low

Detached action object, object exists but is not linked to data in the blob storage SyftError: No blob storage entry exists for uid: e8c3070f1235477ba359090d663f1940
Detached action object, object exists but is not linked to data in the blob storage SyftError: No blob storage entry exists for uid: e8c3070f1235477ba359090d663f1940


```python
class Job:
    id: UID = dd579fa9bcfe4025b2c983d02f2c6190
    status: JobStatus.COMPLETED
    has_parent: False
    result: ActionDataEmpty <<class 'numpy.number'>>
    logs:

0 
JOB COMPLETED
    
```

In [52]:
job_low.wait().get()

17.0